# Phase space MonteCarlo

$$
\bar{p}+p\to X^0[\to J\Psi \pi^0[\to\gamma\gamma]] + \pi^0[\to\gamma\gamma]
$$

$$
\bar{p}+p\to X^0 + \pi^0\to J\Psi + \pi^0 + \pi^0 \to J\Psi + \gamma\gamma + \gamma\gamma
$$

In [1]:
function boost!(v,γ)
    β=sqrt(1-1/γ^2)
    v[1],v[4] = v[1]*γ+v[4]*β*γ, v[1]*β*γ+v[4]*γ
end

function rotateY!(v,cosθ)
    sinθ = sqrt(1-cosθ*cosθ)
    v[2],v[4] = v[2]*cosθ+v[4]*sinθ, -v[2]*sinθ+v[4]*cosθ
end

function rotateZ!(v,ϕ)
    v[2],v[3] = v[2]*cos(ϕ)-v[3]*sin(ϕ), v[3]*cos(ϕ)+v[2]*sin(ϕ)
end

λ(x,y,z) = x^2+y^2+z^2-2*x*y-2*y*z-2*z*x

function masssq(v0)
    p0sq = v0[2]*v0[2]+v0[3]*v0[3]+v0[4]*v0[4];
    msq = v0[1]*v0[1]-p0sq
    return msq
end

z=[0,0,0,1]

function costet(u,v)
    absu=sqrt(u[2]*u[2]+u[3]*u[3]+u[4]*u[4])
    absv=sqrt(v[2]*v[2]+v[3]*v[3]+v[4]*v[4])
    cost=(u[2]*v[2]+u[3]*v[3]+u[4]*v[4])/(absu*absv)
    return acos(cost)*360/(2*pi)
end

function costet2(u,v)
    absu=sqrt(u[1]*u[1]-u[2]*u[2]-u[3]*u[3]-u[4]*u[4])
    absv=sqrt(v[1]*v[1]-v[2]*v[2]-v[3]*v[3]-v[4]*v[4])
    cost=(u[1]*v[1]-u[2]*v[2]-u[3]*v[3]-u[4]*v[4])/(absu*absv)
    return acos(cost)*360/(2*pi)
end

function p3(u)
    absu=sqrt(u[2]*u[2]+u[3]*u[3]+u[4]*u[4])
    return absu
end

function costetz(u)
    return acos(u[4]/u[1])*360/(2*pi)
end

function decay2two(v0,m1sq,m2sq)
    cosθ = 2*rand()-1;
    decay2two(v0,m1sq,m2sq,cosθ)
end

function decay2two(v0,m1sq,m2sq,cosθ)
    p0sq = v0[2]*v0[2]+v0[3]*v0[3]+v0[4]*v0[4];
    msq = v0[1]*v0[1]-p0sq
    (sqrt(msq) < sqrt(m1sq)+sqrt(m2sq)) && return [0,0,0,0], [0,0,0,0]
    p = sqrt(λ(msq,m1sq,m2sq)/(4*msq));
    ϕ = (2*rand()-1)*π;
    sinθ = sqrt(1-cosθ*cosθ);
    v1 = [(msq+m1sq-m2sq)/(2*sqrt(msq)),p*sinθ*sin(ϕ),p*sinθ*cos(ϕ),p*cosθ];
    v2 = [(msq-m1sq+m2sq)/(2*sqrt(msq)),-p*sinθ*sin(ϕ),-p*sinθ*cos(ϕ),-p*cosθ];
    # boost and rotate to the Lab frame
    γ0 = v0[1]/sqrt(msq)
    boost!(v1,γ0); boost!(v2,γ0); 
    # rotate appropriately
    if (p0sq > 0) 
        cosθ0 = v0[4]/sqrt(p0sq)
        rotateY!(v1,cosθ0); rotateY!(v2,cosθ0); 
    end
    ϕ0 = atan2(v0[3],v0[2])
    rotateZ!(v1,ϕ0); rotateZ!(v2,ϕ0);
    return v1,v2
end

decay2two (generic function with 2 methods)

In [2]:
# set constants
const mπ= 0.134; const mπ2 = mπ^2;
const mJΨ = 3.096; const mJΨ2 = mJΨ^2;
const mp = 0.938; const mp2 = mp^2;
const mμ = 0.106; const mμ2 = mμ^2;

# functions to generate random variables
fρ(s,m1sq,m2sq,m3sq,m0sq)=(sqrt(s)>sqrt(m1sq)+sqrt(m2sq)) ? sqrt(λ(s,m1sq,m2sq)*λ(m0sq,s,m3sq))/s : 0;

const dX = collect(linspace((mJΨ+mπ)^2,(4.92-mπ)^2,100));
fX = [fρ(s,mJΨ2,mπ2,mπ2,24.27) for s in 0.5*(dX[1:end-1]+dX[2:end])]; fX /= sum(fX);
[fX[i] = fX[i-1]+fX[i] for i in 2:length(fX)];
const cX = fX;

In [3]:
# X distribution
function randX()
    bi = searchsortedlast(cX,rand())+1
    return dX[bi] + rand()*(dX[bi+1]-dX[bi])
end

randX (generic function with 1 method)

In [4]:
using Plots
#histogram([randX() for i=1:1e5],bins=100, xlab = "mX^2")

In [5]:
function generate()
    p0 = [12,0,0,sqrt(12^2-mp2)] + [mp,0,0,0];
    s0 = p0[1]*p0[1]-p0[2]*p0[2]-p0[3]*p0[3]-p0[4]*p0[4];
    # generate variables
    sX = randX();
    pX,pπ1 = decay2two(p0,sX,mπ2)
    pJΨ,pπ2 = decay2two(pX,mJΨ2,mπ2)
    pγ1,pγ2 = decay2two(pπ1,0,0)
    pγ3,pγ4 = decay2two(pπ2,0,0)
    pμp,pμm = decay2two(pJΨ,mμ2,mμ2)
    return pγ1,pγ2,pγ3,pγ4,pμp,pμm,pJΨ
end

generate (generic function with 1 method)

In [6]:
numberevents=100000    #needs to be integer
gammas=[]
muons=[]
jpsis=[]
for n in 1:numberevents
    vecs=generate()
    for vec in vecs[1:end-3]
        push!(gammas,vec)
    end
    for vec in vecs[end-2:end-1]
        push!(muons,vec)
    end
    for vec in vecs[end:end]
        push!(jpsis,vec)
    end
end

In [20]:
angles=[]
for gamma in gammas[1:end]
    push!(angles,costetz(gamma))
end
energies=[]
for gamma in gammas
    push!(energies,gamma[1])
end
histogram2d(
    angles,
    energies,
    bins=50,
    lab = "", title = "Angle-Energy correlation for photons neutral/4gamma",
    xlab = "theta (deg.)", ylab = "E (GeV)")
hline!([0.8], lc=:red)
vline!([11], lc=:red)

In [15]:
anglesMuon=[]
for muon in muons
    push!(anglesMuon,costetz(muon))
end
p3Muon=[]
for muon in muons
    push!(p3Muon,p3(muon))
end
histogram2d(
    anglesMuon,
    p3Muon,
    bins=100,
    lab = "", title = "Angle-Momentum correlation for muons neutral/4gamma",
    xlab = "theta (deg.)", ylab = "Momentum (GeV)")

In [9]:
#=
anglesjpsi=[]
for jpsi in jpsis[1:end]
    push!(anglesjpsi,costetz(jpsi))
end
energiesjpsi=[]
for jpsi in jpsis
    push!(energiesjpsi,jpsi[1])
end
histogram2d(
    anglesjpsi,
    energiesjpsi,
    bins=50,
    lab = "", title = "Angle-Energy correlation for JΨ's",
    xlab = "theta (deg.)", ylab = "E (GeV)")
=#

# Phase space MonteCarlo

$$
\bar{p}+p\to X^+ + \pi^-\to J\Psi + \pi^+ + \pi^- \to \mu^+\mu^- + \pi^+ + \pi^-
$$

In [10]:
function generateR2()
    p0 = [12,0,0,sqrt(12^2-mp2)] + [mp,0,0,0];
    s0 = p0[1]*p0[1]-p0[2]*p0[2]-p0[3]*p0[3]-p0[4]*p0[4];
    # generate variables
    sX = randX();
    pX,pπm = decay2two(p0,sX,mπ2)
    pJΨ,pπp = decay2two(pX,mJΨ2,mπ2)
    pμp,pμm = decay2two(pJΨ,mμ2,mμ2)
    return pπm,pπp,pμp,pμm,pJΨ
end

generateR2 (generic function with 1 method)

In [11]:
numbereventsR2=100000    #needs to be integer
pionsR2=[]
muonsR2=[]
jpsisR2=[]
for n in 1:numbereventsR2
    vecs=generateR2()
    for vec in vecs[1:end-3]
        push!(pionsR2,vec)
    end
    for vec in vecs[end-2:end-1]
        push!(muonsR2,vec)
    end
    for vec in vecs[end:end]
        push!(jpsisR2,vec)
    end
end

In [16]:
anglesPionR2=[]
for pion in pionsR2
    push!(anglesPionR2,costetz(pion))
end
p3PionR2=[]
for pion in pionsR2
    push!(p3PionR2,p3(pion))
end
histogram2d(
    anglesPionR2,
    p3PionR2,
    bins=100,
    lab = "", title = "Angle-Momentum correlation for pions charged/2pion",
    xlab = "theta (deg.)", ylab = "Momentum (GeV)")

In [17]:
anglesMuonR2=[]
for muon in muonsR2
    push!(anglesMuonR2,costetz(muon))
end
p3MuonR2=[]
for muon in muonsR2
    push!(p3MuonR2,p3(muon))
end
histogram2d(
    anglesMuonR2,
    p3MuonR2,
    bins=100,
    lab = "", title = "Angle-Momentum correlation for muons charged/2pion",
    xlab = "theta (deg.)", ylab = "Momentum (GeV)")